In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
img_o = cv2.imread('co-vay.jpg')
rgb = cv2.cvtColor(img_o, cv2.COLOR_BGR2RGB)
plt.imshow(rgb)

In [ ]:
gray = cv2.cvtColor(img_o, cv2.COLOR_BGR2GRAY)
plt.imshow(gray, cmap='gray')

In [ ]:
thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, 23, 9)
plt.imshow(thresh, cmap='gray')

Lưu ý: Đọc thêm ở đây
https://www.stdio.vn/computer-vision/ky-thuat-grayscale-va-nhi-phan-hoa-anh-adaptive-threshold-Fhjcg

Phương thức adaptiveThreshold
Để chuyển 1 ảnh thành 1 ảnh nhị phân, sử dụng phương thức adaptiveThreshold()

adaptiveThreshold(cv::InputArray src, cv::OutputArray dst, double maxValue,
                  int adaptiveMethod, int thresholdType, int blockSize, double C);
Phân tích

src: hình ảnh gốc (Trong bài viết này là ảnh màu).
dst: ảnh thu được (Trong bài viết này là ảnh nhị phân).
thresh: ngưỡng nhị phân T.
maxValue: giá trị lớn nhất trong ảnh (maxval = 255 đối với ảnh xám).
adaptiveMethod: cách thức nhị phân với ngưỡng động, nó chính là cách tính giá trị ngưỡng nhị phân trong từng vùng cần nhị phân.
thresholdType: kiểu nhị phân. 
blockSize: kích thước của cửa sổ (Region) áp dụng cho việc tính toán ngưỡng động (nên chọn các giá trị %3 = 0 || %5 = 0 || %7 = 0).
C: thông số để bù trừ trong trường hợp ảnh có độ tương phản quá lớn.

In [ ]:
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE,\
                                       cv2.CHAIN_APPROX_SIMPLE)
#nhớ kiểu datatype dạng trên tên là tuple: VD: x, y = (1,2)

print(len(contours))

#sắp xếp sẵn contours theo diện tích
sort_contours = []
for contour in contours:
    area = cv2.contourArea(contour)
    sort_contours.append((contour,area))
sort_contours.sort(key=lambda x: 0 - x[1])

#biến lưu lại giá trị 4 góc của bàn cờ
chessboard_point_conners = None
ui = cv2.cvtColor(img_o.copy(), cv2.COLOR_BGR2RGB)
for contour,_ in sort_contours[0:2]:
    M = cv2.moments(contour) #tính đặc trưng của hình ảnh, tìm hiểu thêm
    if (M["m00"] == 0): continue
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    center = (cX, cY)

    #lấy đường viền bao xung quanh
    contour = cv2.convexHull(contour)
        
    #chu vi của hình
    p = cv2.arcLength(contour, True)
    #bởi vì viền sẽ không thẳng, mà nó sẽ lệnh 1-2%, nên dùng phép toán approxPolyDP để làm thẳng lại
    epsilon = 0.01 * p
    points = cv2.approxPolyDP(contour, epsilon, True)
    
    colorBounded = (np.random.randint(155), np.random.randint(155), np.random.randint(155))
    print('number of points = ', len(points))
    
    if (len(points)) != 4: continue
    
    #vẽ hình vuông bao quanh contour
    #(x,y,w,h) = cv2.boundingRect(contour)
    #cv2.rectangle(ui, (x,y), (x+w,y+h), colorBounded, 2)
    
    cv2.drawContours(ui, [points], -1, colorBounded, 3)
    cv2.circle(ui, center, 6, colorBounded, -1)
    
    
    chessboard_point_conners = points
    # vì chỉ cần 1 hình thôi, nên break code
    break;
    
plt.imshow(ui, cmap='gray')

In [ ]:
#bước tiếp theo: bỏi hình chữ nhật này hơi méo
#nên biến nó thành hình vuông hoàn chỉnh để có thể cắt nó làm 81 miếng hình vuông dễ hơn
#từ khóa để tìm kiếm google: Homography 
print(chessboard_point_conners)
#chessboard_point_conners = expand(chessboard_point_conners)
map_to = np.array([[[0, 0]], [[360,0]], [[360,360]], [[ 0,360]]])
h, mask = cv2.findHomography(chessboard_point_conners, map_to, cv2.RANSAC)

print('h=', h)

In [ ]:
#cắt ảnh ở Image thesh theo h,
#tạo ảnh ROI từ thresh hoặc bất cứ hình nào
roi_chessboard = cv2.warpPerspective(thresh, h, (360, 360))
roi_chessboard.shape

plt.imshow(roi_chessboard, cmap='gray')

In [ ]:
#bởi vì quân cờ số line 1 đang bị cắt, nên phải mở rộng phạm vi cắt
def expand(contour):
    M = cv2.moments(contour) #tính đặc trưng của hình ảnh, tìm hiểu thêm
    if (M["m00"] == 0): return
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    center = (cX, cY)
    
    #tính từ tâm
    for x in range(len(contour)):
        contour[x][0][0] = contour[x][0][0] + (- 20 if contour[x][0][0] < cX else 20)
        contour[x][0][1] = contour[x][0][1] + (- 20 if contour[x][0][1] < cX else 20)
        

In [ ]:
#giờ dùng numpy để chia ra làm 81 hình bên trong
def getImgAt(x_index, y_index):
    startX = x_index * 20
    startY = y_index * 20
    margin = 0
    return roi_chessboard[startY + margin:startY + 40 - margin*2, startX + margin:startX + 40 - margin*2]

plt.imshow(getImgAt(7,0), cmap='gray')

##### Nhận dạng cờ trắng,đen hoặc trống 

In [ ]:
def detectChessValue(x,y):
    #viết sau
    #trả về 0,1, None
    return 1; 

#trả về mảng 2 chiều chess
listResult = [[detectChessValue(x,y) for x in range(0,9)] for y in range(0,9)]
listResult

#### Đọc thêm về các phép xử lý contour:
https://docs.opencv.org/3.4/dd/d49/tutorial_py_contour_features.html